# Running Totals

This will calculate running totals of...
- Points
- Home Points
- Away Points
- Goal Difference
- Home Goal Difference
- Away Goal Difference

#### Import packages

In [1]:
import pandas as pd
import numpy as np
import time

pd.set_option('display.max_columns', None)

#### Read fixtures and teams file

In [2]:
fixtures = pd.read_csv('../../data/csv/fixtures.csv')
teams = pd.read_csv('../../data/csv/teams.csv')

In [3]:
# Check fixtures
fixtures.head()

,id,kickoff_time,team_h,team_a,team_h_score,team_a_score
0,0,2018-08-12 15:00:00+00:00,0,12,0,2
1,1,2018-08-11 14:00:00+00:00,1,4,2,0
2,2,2018-08-11 14:00:00+00:00,8,6,0,2
3,3,2018-08-11 14:00:00+00:00,9,5,0,3
4,4,2018-08-12 12:30:00+00:00,11,18,4,0


### Loop Information

This loop will look through all teams, and then their fixtures. Inside the middle loop, things calculated are...

- Points awarded
- Goal Difference

In [12]:
# Start timing
start_time = time.time()

# Empty DataFrames
df_points = pd.DataFrame()
df_points_home = pd.DataFrame()
df_points_away = pd.DataFrame()
df_points_home_only = pd.DataFrame()
df_points_away_only = pd.DataFrame()

df_goal_difference = pd.DataFrame()
df_goal_difference_home = pd.DataFrame()
df_goal_difference_away = pd.DataFrame()
df_goal_difference_home_only = pd.DataFrame()
df_goal_difference_away_only = pd.DataFrame()

# Loop thorugh all 20 teams
for i in range(20):
    
    # empty points array
    points = np.zeros(38)
    points_home = np.zeros(38)
    points_away = np.zeros(38)
    points_home_only = np.zeros(38)
    points_away_only = np.zeros(38)
    
    # empty goals difference array
    goal_difference = np.zeros(38)
    goal_difference_home = np.zeros(38)
    goal_difference_away = np.zeros(38)
    
    was_home = np.zeros(38, dtype='bool')
    
    # Get home and away fixtures
    a = np.logical_or(fixtures['team_h']==i, fixtures['team_a']==i)
    team_fixtures = (fixtures.loc[a,:]).reset_index(drop=True)
    
    # loop thorugh fixtures. every team has 38 fixtures
    for j in range(38):
        
        if team_fixtures['team_h'][j]==i:
            
            # was_home update
            was_home[j]=True
            
            # if team is at home
            if team_fixtures['team_h_score'][j] > team_fixtures['team_a_score'][j]:
                points[j] = 3
            elif team_fixtures['team_h_score'][j] < team_fixtures['team_a_score'][j]:
                points[j] = 0
            else:
                points[j] = 1
                
            # set home and away points
            points_home[j] = points[j]
            points_away[j] = 0
            
            # goal difference
            goal_difference[j] = team_fixtures['team_h_score'][j] - team_fixtures['team_a_score'][j]
            goal_difference_home[j] = goal_difference[j]
            goal_difference_away[j] = 0
            
        else:
            # if team is away
            if team_fixtures['team_h_score'][j] > team_fixtures['team_a_score'][j]:
                points[j] = 0
            elif team_fixtures['team_h_score'][j] < team_fixtures['team_a_score'][j]:
                points[j] = 3
            else:
                points[j] = 1
            
            # set home and away points
            points_away[j] = points[j]
            points_home[j] = 0
            
            # goal difference
            goal_difference[j] = team_fixtures['team_a_score'][j] - team_fixtures['team_h_score'][j]
            goal_difference_away[j] = goal_difference[j]
            goal_difference_home[j] = 0
    
    # Points
    df_points[teams['short_name'][i]] = pd.Series(np.cumsum(points))
    df_points_home[teams['short_name'][i]] = pd.Series(np.cumsum(points_home))
    df_points_away[teams['short_name'][i]] = pd.Series(np.cumsum(points_away))
    df_points_home_only[teams['short_name'][i]] = pd.Series(np.cumsum(points_home[was_home]))
    df_points_away_only[teams['short_name'][i]] = pd.Series(np.cumsum(points_away[~was_home]))
        
    # Goal differnece
    df_goal_difference[teams['short_name'][i]] = pd.Series(np.cumsum(goal_difference))
    df_goal_difference_home[teams['short_name'][i]] = pd.Series(np.cumsum(goal_difference_home))
    df_goal_difference_away[teams['short_name'][i]] = pd.Series(np.cumsum(goal_difference_away))
    df_goal_difference_home_only[teams['short_name'][i]] = pd.Series(np.cumsum(goal_difference_home[was_home]))
    df_goal_difference_away_only[teams['short_name'][i]] = pd.Series(np.cumsum(goal_difference_away[~was_home]))
    
    
print("Time Taken: %s seconds" % (time.time() - start_time))

Time Taken: 0.1711592674255371 seconds


In [15]:
df_points.to_csv(r'../../data/csv/team_running_points.csv', index=False, index_label=False)
print(df_points.shape)

(38, 20)


In [16]:
df_points_home.to_csv(r'../../data/csv/team_running_points_home.csv', index=False, index_label=False)
print(df_points_home.shape)

(38, 20)


In [7]:
df_points_away.to_csv(r'../../data/csv/team_running_points_away.csv', index=False, index_label=False)
df_points_away.head(38)

,ARS,BOU,BHA,BUR,CAR,CHE,CRY,EVE,FUL,HUD,LEI,LIV,MCI,MUN,NEW,SOU,TOT,WAT,WHU,WOL
0,0.0,0.0,0.0,1.0,0.0,3.0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
1,0.0,3.0,0.0,1.0,0.0,3.0,3.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0,0.0,3.0,3.0,0.0,0.0
2,0.0,3.0,0.0,1.0,1.0,6.0,3.0,2.0,0.0,0.0,3.0,3.0,4.0,0.0,1.0,0.0,6.0,3.0,0.0,0.0
3,3.0,3.0,0.0,1.0,1.0,6.0,3.0,2.0,1.0,1.0,3.0,6.0,4.0,3.0,1.0,3.0,6.0,3.0,0.0,3.0
4,6.0,3.0,1.0,1.0,1.0,6.0,6.0,2.0,1.0,1.0,3.0,9.0,4.0,6.0,1.0,3.0,6.0,3.0,3.0,3.0
5,6.0,3.0,1.0,1.0,1.0,7.0,6.0,2.0,1.0,1.0,3.0,9.0,7.0,6.0,2.0,3.0,9.0,4.0,3.0,4.0
6,6.0,3.0,1.0,4.0,1.0,7.0,6.0,2.0,1.0,1.0,6.0,10.0,7.0,6.0,2.0,3.0,12.0,4.0,3.0,4.0
7,9.0,6.0,1.0,4.0,1.0,10.0,6.0,5.0,1.0,2.0,6.0,10.0,8.0,6.0,2.0,3.0,12.0,4.0,3.0,7.0
8,9.0,6.0,4.0,4.0,1.0,10.0,6.0,5.0,1.0,2.0,6.0,13.0,8.0,7.0,2.0,4.0,15.0,7.0,3.0,7.0
9,10.0,9.0,4.0,4.0,1.0,13.0,6.0,5.0,1.0,2.0,6.0,13.0,11.0,7.0,3.0,4.0,15.0,7.0,4.0,7.0


In [8]:
df_goal_difference.to_csv(r'../../data/csv/team_running_goal_difference.csv', index=False, index_label=False)
df_goal_difference.head(38)

,ARS,BOU,BHA,BUR,CAR,CHE,CRY,EVE,FUL,HUD,LEI,LIV,MCI,MUN,NEW,SOU,TOT,WAT,WHU,WOL
0,-2.0,2.0,-2.0,0.0,-2.0,3.0,2.0,0.0,-2.0,-3.0,-1.0,4.0,2.0,1.0,-1.0,0.0,1.0,2.0,-4.0,0.0
1,-3.0,3.0,-1.0,-2.0,-2.0,4.0,0.0,1.0,-4.0,-8.0,1.0,6.0,7.0,0.0,-1.0,-1.0,3.0,4.0,-5.0,-2.0
2,-1.0,3.0,-2.0,-4.0,-2.0,5.0,-1.0,1.0,-2.0,-8.0,2.0,7.0,7.0,-3.0,-2.0,-2.0,6.0,5.0,-7.0,-2.0
3,0.0,1.0,-2.0,-6.0,-3.0,7.0,-3.0,1.0,-2.0,-8.0,1.0,8.0,8.0,-1.0,-3.0,0.0,5.0,6.0,-8.0,-1.0
4,1.0,3.0,-2.0,-7.0,-6.0,10.0,-2.0,-1.0,-5.0,-9.0,-1.0,9.0,11.0,0.0,-4.0,0.0,4.0,5.0,-6.0,0.0
5,3.0,-1.0,-3.0,-3.0,-11.0,10.0,-2.0,-3.0,-5.0,-11.0,1.0,12.0,16.0,0.0,-4.0,-3.0,5.0,5.0,-6.0,0.0
6,5.0,0.0,-5.0,-2.0,-12.0,10.0,-3.0,0.0,-8.0,-13.0,3.0,12.0,18.0,-2.0,-6.0,-5.0,7.0,3.0,-4.0,2.0
7,9.0,4.0,-4.0,-2.0,-13.0,13.0,-4.0,1.0,-12.0,-13.0,2.0,12.0,18.0,-1.0,-7.0,-8.0,8.0,-1.0,-5.0,3.0
8,11.0,4.0,-3.0,-7.0,-11.0,13.0,-6.0,3.0,-14.0,-14.0,0.0,13.0,23.0,-1.0,-8.0,-8.0,9.0,1.0,-6.0,1.0
9,11.0,7.0,-2.0,-11.0,-14.0,17.0,-6.0,2.0,-17.0,-17.0,0.0,16.0,24.0,0.0,-8.0,-8.0,8.0,4.0,-6.0,0.0


In [9]:
df_goal_difference_home.to_csv(r'../../data/csv/team_running_goal_difference_home.csv', index=False, index_label=False)
df_goal_difference_home.head(38)

,ARS,BOU,BHA,BUR,CAR,CHE,CRY,EVE,FUL,HUD,LEI,LIV,MCI,MUN,NEW,SOU,TOT,WAT,WHU,WOL
0,-2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,-3.0,0.0,4.0,0.0,1.0,-1.0,0.0,0.0,2.0,0.0,0.0
1,-2.0,2.0,1.0,-2.0,0.0,1.0,-2.0,1.0,-2.0,-3.0,2.0,4.0,5.0,1.0,-1.0,0.0,2.0,2.0,-1.0,0.0
2,0.0,2.0,1.0,-2.0,0.0,1.0,-2.0,1.0,0.0,-3.0,2.0,5.0,5.0,-2.0,-2.0,-1.0,2.0,3.0,-1.0,0.0
3,0.0,2.0,1.0,-4.0,-1.0,3.0,-4.0,1.0,0.0,-3.0,1.0,5.0,6.0,-2.0,-2.0,-1.0,2.0,4.0,-2.0,0.0
4,0.0,4.0,1.0,-4.0,-1.0,6.0,-4.0,-1.0,0.0,-4.0,1.0,5.0,9.0,-2.0,-3.0,-1.0,1.0,3.0,-2.0,1.0
5,2.0,4.0,0.0,0.0,-6.0,6.0,-4.0,-1.0,0.0,-4.0,3.0,8.0,9.0,-2.0,-3.0,-1.0,1.0,3.0,-2.0,1.0
6,4.0,5.0,0.0,0.0,-7.0,6.0,-4.0,2.0,0.0,-6.0,3.0,8.0,11.0,-2.0,-5.0,-1.0,1.0,3.0,0.0,3.0
7,4.0,5.0,1.0,0.0,-7.0,6.0,-5.0,2.0,-4.0,-6.0,2.0,8.0,11.0,-1.0,-5.0,-4.0,2.0,-1.0,0.0,3.0
8,6.0,5.0,1.0,0.0,-5.0,6.0,-5.0,4.0,-4.0,-7.0,2.0,8.0,16.0,-1.0,-6.0,-4.0,2.0,-1.0,-1.0,1.0
9,6.0,5.0,2.0,-4.0,-5.0,6.0,-5.0,4.0,-7.0,-7.0,2.0,11.0,16.0,0.0,-6.0,-4.0,1.0,2.0,-1.0,1.0


In [10]:
df_goal_difference_away.to_csv(r'../../data/csv/team_running_goal_difference_away.csv', index=False, index_label=False)
df_goal_difference_away.head(38)

,ARS,BOU,BHA,BUR,CAR,CHE,CRY,EVE,FUL,HUD,LEI,LIV,MCI,MUN,NEW,SOU,TOT,WAT,WHU,WOL
0,0.0,0.0,-2.0,0.0,-2.0,3.0,2.0,0.0,0.0,0.0,-1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,-4.0,0.0
1,-1.0,1.0,-2.0,0.0,-2.0,3.0,2.0,0.0,-2.0,-5.0,-1.0,2.0,2.0,-1.0,0.0,-1.0,1.0,2.0,-4.0,-2.0
2,-1.0,1.0,-3.0,-2.0,-2.0,4.0,1.0,0.0,-2.0,-5.0,0.0,2.0,2.0,-1.0,0.0,-1.0,4.0,2.0,-6.0,-2.0
3,0.0,-1.0,-3.0,-2.0,-2.0,4.0,1.0,0.0,-2.0,-5.0,0.0,3.0,2.0,1.0,-1.0,1.0,3.0,2.0,-6.0,-1.0
4,1.0,-1.0,-3.0,-3.0,-5.0,4.0,2.0,0.0,-5.0,-5.0,-2.0,4.0,2.0,2.0,-1.0,1.0,3.0,2.0,-4.0,-1.0
5,1.0,-5.0,-3.0,-3.0,-5.0,4.0,2.0,-2.0,-5.0,-7.0,-2.0,4.0,7.0,2.0,-1.0,-2.0,4.0,2.0,-4.0,-1.0
6,1.0,-5.0,-5.0,-2.0,-5.0,4.0,1.0,-2.0,-8.0,-7.0,0.0,4.0,7.0,0.0,-1.0,-4.0,6.0,0.0,-4.0,-1.0
7,5.0,-1.0,-5.0,-2.0,-6.0,7.0,1.0,-1.0,-8.0,-7.0,0.0,4.0,7.0,0.0,-2.0,-4.0,6.0,0.0,-5.0,0.0
8,5.0,-1.0,-4.0,-7.0,-6.0,7.0,-1.0,-1.0,-10.0,-7.0,-2.0,5.0,7.0,0.0,-2.0,-4.0,7.0,2.0,-5.0,0.0
9,5.0,2.0,-4.0,-7.0,-9.0,11.0,-1.0,-2.0,-10.0,-10.0,-2.0,5.0,8.0,0.0,-2.0,-4.0,7.0,2.0,-5.0,-1.0
